In [ ]:
# @title Inspect the schema of qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.order_item
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.order_item')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Inspect the schema of qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.orders
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.orders')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Inspect the schema of qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.menu
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.menu')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Setup bigquery client and formatting
from google.cloud import bigquery
from google.colab import data_table

project = 'qwiklabs-gcp-03-b5f1eb4011f7' # Project ID
client = bigquery.Client(project=project)
data_table.enable_dataframe_formatter()
# @title Executes the query, using client from previous cell
sql_SQL = '''# prompt: Join these data sources

# prompt: Join these data sources

SELECT
menu.menu_id,
menu.company_id,
menu.item_name,
menu.item_price,
menu.item_description,
menu.item_size,
order_item.order_item_id,
order_item.order_id,
order_item.quantity,
order_item.item_price,
order_item.item_total,
orders.location_id,
orders.customer_id,
orders.order_datetime,
orders.order_completion_datetime
FROM
`qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.menu` AS menu
INNER JOIN
`qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.order_item` AS order_item
ON
menu.menu_id = order_item.menu_id
INNER JOIN
`qwiklabs-gcp-03-b5f1eb4011f7.coffee_on_wheels.orders` AS orders
ON
order_item.order_id = orders.order_id;'''
query_SQL = client.query(sql_SQL)

# store the destination to reference in future cells
query_destination_SQL = query_SQL.destination.project + '.' + query_SQL.destination.dataset_id + '.' + query_SQL.destination.table_id

#  Render the query results
job_SQL= client.get_job(query_SQL.job_id)
df_SQL = job_SQL.to_dataframe()
df_SQL


In [ ]:

# @title Executes the query, using client from previous cell
sql_SQL_2 = '''# prompt: Identify the top 10 items by total revenue and include the menu_id, item_nam, item_size, and total_revenue fields.

SELECT
  t1.menu_id,
  t1.item_name,
  t1.item_size,
  SUM(t1.item_total) AS total_revenue
FROM
  ''' + query_destination_SQL + ''' AS t1
GROUP BY
  1,
  2,
  3
ORDER BY
  total_revenue DESC
LIMIT
  10;'''
query_SQL_2 = client.query(sql_SQL_2)

# store the destination to reference in future cells
query_destination_SQL_2 = query_SQL_2.destination.project + '.' + query_SQL_2.destination.dataset_id + '.' + query_SQL_2.destination.table_id

#  Render the query results
job_SQL_2= client.get_job(query_SQL_2.job_id)
df_SQL_2 = job_SQL_2.to_dataframe()
df_SQL_2


In [ ]:
# @title Create a vertical bar chart displaying total revenue. Include the item name on the x-axis and the total revenue on the y-axis in the results. Start with the location with the highest revenue. Stack the results by item size.

import altair as alt

chart1 = alt.Chart(data=df1, mark={
  "type": "bar",
  "tooltip": True
}).encode(
  x={
  "field": "item_name",
  "type": "nominal",
  "title": "Item Name",
  "sort": {
    "field": "total_revenue",
    "order": "descending"
  },
  "axis": {
    "labelOverlap": True
  }
},
  y={
  "field": "total_revenue",
  "type": "quantitative",
  "title": "Total Revenue",
  "axis": {
    "labelOverlap": True
  }
},
  color={
  "field": "item_size",
  "type": "nominal",
  "title": "Item Size"
},
  
)
chart1


* The Brewhaha Bonanza generated the highest revenue at $1,480.34, representing 21.88% of total revenue, while Espresso Infused Jelly Donuts brought in the lowest at $615.41, or 9.1% of total revenue.
	* 
* Sales from items with a "Large" size contributed the most to total revenue at 48.33%, while "Medium" sized items generated the lowest revenue at 10.16%.
	* 
* Interestingly, no sales data is available for "Coffee-infused Avocado Toast" and "Espresso Cheesecake," indicating a potential gap in product offerings or data capture.
	* 
* The combination of "Brewhaha Bonanza" and "Large" size proved to be a successful sales driver, generating 11.72% of total revenue. Conversely, "Espresso Infused Jelly Donuts" without a specified size recorded the lowest revenue at 9.1%, suggesting a potential need for size options or further analysis.
	* 

In [ ]:

# @title Executes the query, using client from previous cell
sql_SQL_1 = '''# prompt: From the joined table, only consider orders from 2024. Calculate the total revenue for each menu item. In the results, display the menu_id, the item name, the item size, and the total revenue (rounded as only two decimal places). Order results with total revenue in descending order.

SELECT
t1.menu_id,
t1.item_name,
t1.item_size,
ROUND(SUM(t1.item_total), 2) AS total_revenue
FROM
''' + query_destination_SQL + ''' AS t1
WHERE
EXTRACT(YEAR
FROM
   t1.order_datetime) = 2024
GROUP BY
1,
2,
3
ORDER BY
total_revenue DESC;'''
query_SQL_1 = client.query(sql_SQL_1)

# store the destination to reference in future cells
query_destination_SQL_1 = query_SQL_1.destination.project + '.' + query_SQL_1.destination.dataset_id + '.' + query_SQL_1.destination.table_id

#  Render the query results
job_SQL_1= client.get_job(query_SQL_1.job_id)
df_SQL_1 = job_SQL_1.to_dataframe()
df_SQL_1


In [ ]:

# @title Executes the query, using client from previous cell
sql_SQL_3 = '''# prompt: Find two items with the same total revenue. Within the results, display the item names, item size, and total revenue. Limit your response to only two items.

SELECT
t1.item_name,
t1.item_size,
t1.total_revenue
FROM
''' + query_destination_SQL_1 + ''' AS t1
WHERE
t1.total_revenue IN(
SELECT
   t2.total_revenue
FROM
   ''' + query_destination_SQL_1 + ''' AS t2
GROUP BY
   1
HAVING
   COUNT(t2.total_revenue) > 1 )
LIMIT
2;'''
query_SQL_3 = client.query(sql_SQL_3)

# store the destination to reference in future cells
query_destination_SQL_3 = query_SQL_3.destination.project + '.' + query_SQL_3.destination.dataset_id + '.' + query_SQL_3.destination.table_id

#  Render the query results
job_SQL_3= client.get_job(query_SQL_3.job_id)
df_SQL_3 = job_SQL_3.to_dataframe()
df_SQL_3
